In [1]:
import pandas as pd 
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

In [2]:
# Pandas için varsayılan plot uygulamasını plotly yapıyoruz.
pd.options.plotting.backend = "plotly"
warnings.simplefilter("ignore")

In [3]:
train = pd.read_csv("/kaggle/input/install-future-program-ankara-hackathon/train.csv")

In [4]:
train["DATETIME"] = pd.to_datetime(train["DATETIME"])

### Anomaly Noktaları Bulmak İçin IQR(Box Plot) Tekniğini Kullandım. Bu Teknik, Girilen Yüzdelik Değerler İçin(q1, q3) low ve up Değerleri Buluyor. Bu Değerlerden Düşük veya Yüksek Olan Değerler **Anomaly** Değerlerdir. 

In [5]:
def outlier_thresholds(df, col_name, q1=0.25, q3=0.75):
    quartile1 = df[col_name].quantile(q1)
    quartile3 = df[col_name].quantile(q3)
    iqr = quartile3 - quartile1
    up = quartile3 + 1.5 * iqr
    low = quartile1 - 1.5 * iqr
    return low, up

In [6]:
def check_outlier(df, col_name):
    low, up = outlier_thresholds(df, col_name)
    return True if df[(df[col_name] > up) | (df[col_name] < low)].any(axis = None) else False

In [7]:
def replace_with_thresholds(df, veriable):
    df = df.copy()
    low, up = outlier_thresholds(df, veriable)
    df.loc[(df[veriable] < low), veriable] = low
    df.loc[(df[veriable] > up), veriable] = up
    return df

In [8]:
def preproccesing_df(df):
    df = df.copy()
    res_df = pd.DataFrame()
    for i in df["SERVER"].unique():
        temp_df = df[df["SERVER"] == i]
        temp_df = temp_df.groupby("DATETIME").agg({"CPULOAD":"mean"}).reset_index()
        temp_df["SERVER"] = i
        res_df = pd.concat([res_df, temp_df])
    
    return res_df

In [9]:
prep_train = preproccesing_df(train)

In [10]:
df_301 = prep_train[prep_train["SERVER"] == "CIHAZ_301"]

In [11]:
print(outlier_thresholds(df_301, "CPULOAD"))

(-5.393020833333336, 19.55947916666667)


### Değerlerimiz 0'dan düşük olmadığı için up değer outlier için dikkate alınmıştır.

In [12]:
threshold = outlier_thresholds(df_301, "CPULOAD")[1]

In [13]:
df_301.loc[:,"OUTLIER"] = df_301["CPULOAD"] > threshold

### Server 301'e ait 'CPULOAD' Scatter Plot

In [14]:
fig = px.scatter(df_301,
                 x="DATETIME",
                 y="CPULOAD",
                 color="CPULOAD",
                 color_continuous_scale="RdBu",
                 title="Server 301 Scatter Plot")
fig.show()

### SERVER 301'e ait 'CPULOAD' Box Plot

In [15]:
df_301.boxplot(y="CPULOAD")

### SERVER 301'e ait 'CPULOAD' Outlier Grafiği

In [16]:
fig = df_301.plot.scatter(x="DATETIME",
                    y="CPULOAD",
                    title="SERVER 301 CPULOAD OUTLIER DETECTION",
                    color="OUTLIER",
                    symbol="OUTLIER")

fig.show()

### SERVER 301'e ait 'CPULOAD' Outlier Noktalarının Kaydedilmesi

In [17]:
server_301_outliers = df_301[df_301["OUTLIER"] == True]
server_301_outliers

,DATETIME,CPULOAD,SERVER,OUTLIER
1712,2020-04-10 22:00:00+03:00,19.978333,CIHAZ_301,True
5889,2020-05-24 11:15:00+03:00,20.239583,CIHAZ_301,True
5890,2020-05-24 11:30:00+03:00,21.688750,CIHAZ_301,True
5891,2020-05-24 11:45:00+03:00,23.526250,CIHAZ_301,True
5892,2020-05-24 12:00:00+03:00,24.563333,CIHAZ_301,True
5893,2020-05-24 12:15:00+03:00,22.832083,CIHAZ_301,True
5894,2020-05-24 12:30:00+03:00,23.965833,CIHAZ_301,True
5895,2020-05-24 12:45:00+03:00,22.315000,CIHAZ_301,True
5896,2020-05-24 13:00:00+03:00,21.800833,CIHAZ_301,True
5897,2020-05-24 13:15:00+03:00,21.304583,CIHAZ_301,True


In [18]:
with open("server_301_outlier_df.pckl", "wb") as f:
    pickle.dump(server_301_outliers, f)
    print("Dosya Başarıyla Kaydedildi!")

Dosya Başarıyla Kaydedildi!


## SERVER 305'e ait Analizler

In [19]:
df_305 = prep_train[prep_train["SERVER"] == "CIHAZ_305"]

In [20]:
print(outlier_thresholds(df_305, "CPULOAD"))

(-13.515677083333326, 38.512239583333326)


In [21]:
threshold = outlier_thresholds(df_305, "CPULOAD")[1]

In [22]:
df_305.loc[:,"OUTLIER"] = df_305["CPULOAD"] > threshold

### Server 305'e ait 'CPULOAD' Scatter Plot

In [23]:
fig = px.scatter(df_305, 
                 x="DATETIME", 
                 y="CPULOAD", 
                 color="CPULOAD", 
                 color_continuous_scale="RdBu",
                 title="Server 305 Scatter Plot")
fig.show()

### SERVER 305'e ait 'CPULOAD' Box Plot

In [24]:
df_305.boxplot(y="CPULOAD")

### SERVER 305'e ait 'CPULOAD' Outlier Grafiği

In [25]:
fig = df_305.plot.scatter(x="DATETIME",
                          y="CPULOAD",
                          title="SERVER 305 CPULOAD OUTLIER DETECTION",
                          color="OUTLIER",
                          symbol="OUTLIER")

fig.show()

### SERVER 305'e ait 'CPULOAD' Outlier Noktalarının Kaydedilmesi

In [26]:
server_305_outliers = df_305[df_305["OUTLIER"] == True]
server_305_outliers

,DATETIME,CPULOAD,SERVER,OUTLIER
5886,2020-05-24 11:00:00+03:00,38.970000,CIHAZ_305,True
5887,2020-05-24 11:15:00+03:00,47.099167,CIHAZ_305,True
5888,2020-05-24 11:30:00+03:00,46.526250,CIHAZ_305,True
5889,2020-05-24 11:45:00+03:00,50.281667,CIHAZ_305,True
5890,2020-05-24 12:00:00+03:00,53.384583,CIHAZ_305,True
5891,2020-05-24 12:15:00+03:00,50.530417,CIHAZ_305,True
5892,2020-05-24 12:30:00+03:00,50.867083,CIHAZ_305,True
5893,2020-05-24 12:45:00+03:00,48.047500,CIHAZ_305,True
5894,2020-05-24 13:00:00+03:00,46.573750,CIHAZ_305,True
5895,2020-05-24 13:15:00+03:00,45.642500,CIHAZ_305,True


In [27]:
with open("server_305_outlier_df.pckl", "wb") as f:
    pickle.dump(server_305_outliers, f)
    print("Dosya Başarıyla Kaydedildi!")

Dosya Başarıyla Kaydedildi!
